In [29]:
from patchify import patchify
import pandas as pd
import os
import numpy as np
import nibabel as nib                                                     # Read / write access to some common neuroimaging file formats
import itk                                                                # Jupyter widgets to visualize images in 2D and 3D
import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import matplotlib.pyplot as plt
import cv2


# Load source images dataframe

In [2]:
root_folder = '../../datasets/ADDA/resampled/'
root_images_folder = '../../datasets/ADDA/resampled/images'
root_labels_folder = '../../datasets/ADDA/resampled/labels'

In [3]:
images_df = pd.read_csv(os.path.join(root_folder, 'images_info.csv'))
images_df.info()
                        

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   File Name  684 non-null    object 
 1   Institute  684 non-null    object 
 2   spacing    684 non-null    object 
 3   size       684 non-null    object 
 4   modality   684 non-null    object 
 5   spacing_x  684 non-null    float64
 6   spacing_y  684 non-null    float64
 7   spacing_z  684 non-null    float64
 8   size_x     684 non-null    float64
 9   size_y     684 non-null    float64
 10  size_z     684 non-null    float64
dtypes: float64(6), object(5)
memory usage: 58.9+ KB


In [4]:
images_df.head()

,File Name,Institute,spacing,size,modality,spacing_x,spacing_y,spacing_z,size_x,size_y,size_z
0,MDA-111__CT.nii.gz,MDA,"(1.0, 1.0, 1.0)","(500, 500, 975)",CT,1.0,1.0,1.0,500.0,500.0,975.0
1,MDA-126__CT.nii.gz,MDA,"(1.0, 1.0, 1.0)","(500, 500, 415)",CT,1.0,1.0,1.0,500.0,500.0,415.0
2,MDA-005__CT.nii.gz,MDA,"(1.0, 1.0, 1.0)","(500, 500, 873)",CT,1.0,1.0,1.0,500.0,500.0,873.0
3,CHUS-048__CT.nii.gz,CHUS,"(1.0, 1.0, 1.0)","(576, 576, 401)",CT,1.0,1.0,1.0,576.0,576.0,401.0
4,MDA-032__CT.nii.gz,MDA,"(1.0, 1.0, 1.0)","(498, 498, 873)",CT,1.0,1.0,1.0,498.0,498.0,873.0


# Load single image

In [39]:
img_name = 'MDA-079'

ct_obj = nib.load(os.path.join(root_images_folder, f'{img_name}__CT.nii.gz'))
pt_obj = nib.load(os.path.join(root_images_folder, f'{img_name}__PT.nii.gz'))
mask_obj = nib.load(os.path.join(root_labels_folder, f'{img_name}.nii.gz'))

In [40]:
ct_image_data = ct_obj.get_fdata()
pt_image_data = pt_obj.get_fdata()
mask_data = mask_obj.get_fdata()

In [41]:
# Get the image shape and print it out
height, width, depth = ct_image_data.shape
print(f"The image object has the following dimensions: height: {height}, width:{width}, depth:{depth}")

The image object has the following dimensions: height: 500, width:500, depth:415


In [44]:
# only keep GTVp=1 ignore GTVn=2
mask_data = (mask_data == 1)

In [45]:
def apply_window(img, width=500, center=40):
    # np.interp(a, (a.min(), a.max()), (-1, +1))

    # convert below window to black
    img[img<(center-(width/2))]=center-(width/2)
    # convert above window to white
    img[img>(center+(width/2))]=center+(width/2)

    # normalize image
    img_scaled = np.interp(img, (img.min(), img.max()), (0, +1))
    # print(img_scaled.shape)
    # print(np.min(img_scaled), np.max(img_scaled))
    return img

In [46]:
# Define a function to visualize the data
def explore_3dimage(layer):
#     plt.figure(figsize=(20, 10))
#     channel = 3
#     plt.imshow(ct_image_data[:, :, layer], cmap='gray');
#     plt.title('Explore Layers of Brain CT', fontsize=20)
#     plt.axis('off')
    fig, ax = plt.subplots(ncols=3, figsize=(20,20))

    ax[0].imshow(apply_window(ct_image_data[:,:,layer]), cmap='gray')
    ax[0].contour(mask_data[:,:,layer], colors='red')
    
    ax[1].imshow(pt_image_data[:,:,layer], cmap='gray')
    ax[1].contour(mask_data[:,:,layer],  colors='red')
    
    ax[2].imshow(mask_data[:,:,layer], cmap='gray')

    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dimage, layer=(0, ct_image_data.shape[2] - 1))

interactive(children=(IntSlider(value=207, description='layer', max=414), Output()), _dom_classes=('widget-int…

<function __main__.explore_3dimage(layer)>

# try patchify

In [30]:
ct_image_data.shape

(500, 500, 415)

In [31]:
pSize = 256
st = 128
ct_patches = patchify(ct_image_data,(pSize,pSize,pSize), step=st)
pt_patches = patchify(pt_image_data,(pSize,pSize,pSize), step=st)
mask_patches = patchify(mask_data,(pSize,pSize,pSize), step=st)

In [32]:
ct_patches.shape

(2, 2, 2, 256, 256, 256)

In [33]:
xx = (mask_patches != 0)
indices = np.where(xx)
non_zero_indices = list(zip(*indices))
for i, index_tuple in enumerate(non_zero_indices):
    print(f"Non-zero value {i+1} at indices: {index_tuple}")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [34]:
y,x,z = 0,1,1
ct_test = ct_patches[x][y][z]
pt_test = pt_patches[x][y][z]
mask_test = mask_patches[x][y][z]

In [35]:
# Define a function to visualize the data
def explore_3dpatch(layer):
 
    fig, ax = plt.subplots(ncols=3, figsize=(5,5))

    # ax[0].imshow(apply_window(ct[:,:,layer]), cmap='gray')
    ax[0].imshow(apply_window(ct_test[:,:,layer]), cmap='gray')
    ax[0].contour(mask_test[:,:,layer], colors='red')
    
    ax[1].imshow(pt_test[:,:,layer], cmap='gray')
    ax[1].contour(mask_test[:,:,layer],  colors='red')
    
    ax[2].imshow(mask_test[:,:,layer], cmap='gray')


    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dpatch, layer=(0, ct_test.shape[2] - 1))

interactive(children=(IntSlider(value=127, description='layer', max=255), Output()), _dom_classes=('widget-int…

<function __main__.explore_3dpatch(layer)>

In [196]:
ct_test.shape

(64, 64, 64)

In [197]:

def concat_pet_ct(ct, pt):
    assert ct.shape == pt.shape
    # Create an empty array to store the concatenated array
    concatenated_array = np.empty((ct.shape[0], ct.shape[1], 2*ct.shape[2]))
    # Iterate over the two arrays and concatenate them in an interchangeable order along the z axis
    for i in range(64):
        concatenated_array[:, :, 2*i] = ct[:, :, i]
        concatenated_array[:, :, 2*i + 1] = pt[:, :, i]

    return concatenated_array
    

dd = concat_pet_ct(ct_test, pt_test)
mm = concat_pet_ct(mask_test, mask_test)
print(dd.shape)
print(mm.shape)


# # Print the shape of the concatenated array
# print(concatenated_array.shape)

(64, 64, 128)
(64, 64, 128)


In [199]:
# Define a function to visualize the data
def explore_3dconcat(layer):
 
    fig, ax = plt.subplots(ncols=1, figsize=(5,5))

    # ax[0].imshow(apply_window(ct[:,:,layer]), cmap='gray')
    ax.imshow(dd[:,:,layer], cmap='gray')
    ax.contour(mm[:,:,layer])
    


    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dconcat, layer=(0, dd.shape[2] - 1))

interactive(children=(IntSlider(value=63, description='layer', max=127), Output()), _dom_classes=('widget-inte…

<function __main__.explore_3dconcat(layer)>

# Sandbox

In [10]:
import numpy as np

# Create an empty NumPy array
empty_array = np.array([])


In [13]:
# Create some NumPy arrays to append
array1 = np.array([1, 2, 3])
array2 = np.array([4, 5, 6])

# Append the arrays to the empty array
empty_array = np.concatenate(empty_array, array1)
empty_array = np.concatenate(empty_array, array2)


TypeError: only integer scalar arrays can be converted to a scalar index

In [12]:
empty_array.shape

(6,)

In [27]:
empty = []

arr1 = np.zeros([1, 500,500,413], dtype=float)
arr2 = np.zeros([1, 500,500,423], dtype=float)
print(arr1.shape, arr2.shape)

empty.append(arr1)
empty.append(arr2)


np.array(empty).shape
             

(1, 500, 500, 413) (1, 500, 500, 423)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 4 dimensions. The detected shape was (2, 1, 500, 500) + inhomogeneous part.